## RDDs

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
# Define the rdd for testing

rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
# filter by date
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [8]:
# Define variables and return a tuple
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    # Key tuple
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    # Value tuple
    value = (amount, count)

    # Return a tuple
    return (key, value)

In [9]:
# Calculate the revenue and returns a tuple
def calculate_revenue(left_value, right_value):
    # unpack values
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    # Sum it
    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    # Return a tuple
    return (output_amount, output_count)

In [10]:
from collections import namedtuple

In [11]:
# This is how you provide column names
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [12]:
# Unwrap each row 
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [13]:
from pyspark.sql import types

In [14]:
# Define schema
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [15]:
# This are the steps the rdd will follow to be saved as a df
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [17]:
# Save results into a parquet file
df_result \
    .write.parquet('tmp/green-revenue', mode='overwrite')

In [19]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [20]:
import pandas as pd

In [21]:
rows = duration_rdd.take(10)

In [25]:
rows[0]

Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77)

In [22]:
df = pd.DataFrame(rows, columns=columns)

In [23]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [76]:
# Simulates a ml prediction model

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [98]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [102]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [104]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|             12.95|
|             31.25|
|              14.0|
|             12.75|
|               0.1|
|             11.05|
|11.299999999999999|
|54.349999999999994|
|             15.25|
|             91.75|
|             12.25|
|               3.1|
|               7.5|
|11.899999999999999|
| 78.89999999999999|
|              4.45|
|              23.2|
|              4.85|
|              6.65|
|              15.1|
+------------------+
only showing top 20 rows

